In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential,layers
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow.keras.layers import Conv2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping,LearningRateScheduler
from tensorflow.keras.optimizers.experimental import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-03-08 12:49:11.071913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
import os
from PIL import Image

In [32]:
label_df = pd.read_csv('/Users/justinpak/code/justinpakzad/head_hunter/head_hunter/raw_data/kaggle_crowd_counter_data/labels.csv')

In [33]:
label_df.columns = ['id','people']

In [34]:
img = np.load('/Users/justinpak/code/justinpakzad/head_hunter/head_hunter/raw_data/kaggle_crowd_counter_data/images.npy')

In [35]:
label_df.head()

,id,people
0,1,35
1,2,41
2,3,41
3,4,44
4,5,41


In [36]:
img.shape

(2000, 480, 640, 3)

In [37]:
labels = np.array(label_df['people'])

In [38]:
X_train,X_test,y_train,y_test = train_test_split(img,labels,test_size=0.1)

In [40]:
print(X_train.shape[0])
print(X_test.shape[0])

1800
200


In [41]:
X_train,X_test = X_train/255,X_test/255

In [42]:
def init_model():
    model = Sequential()
    model.add(layers.Conv2D(64, (3,3), input_shape=(480,640,3), activation='relu')),
    model.add(layers.MaxPool2D(2,2)),
    model.add(layers.Conv2D(128, (3,3),activation='relu')),
    model.add(layers.MaxPool2D(2,2)),
    
    model.add(layers.Flatten()),
    model.add(layers.Dense(128, activation='relu')),
    model.add(layers.Dense(1))
    
    
    model.compile(loss='huber_loss',optimizer='adam',metrics=['mae'])
    
    return model

In [43]:
model = init_model()

In [44]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 478, 638, 64)      1792      
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 239, 319, 64)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 237, 317, 128)     73856     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 118, 158, 128)    0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 2386432)           0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [ ]:

lr_mon = LearningRateScheduler(
                lambda epochs : 1e-8 * 10 ** (epochs/20))

history = model.fit(X_train, y_train, validation_data=[X_test, y_test], epochs=50, batch_size=32, callbacks=[lr_mon])

Epoch 1/50
